In [1]:
import itertools # 是 Python 的內建模組，提供了一組用於處理迭代器的函數和工具。
                 # 它包含了各種用於高效處理迭代器的函數，可以幫助我們編寫更簡潔、高效的代碼。
import sys # 是 Python 的內建模組，提供了與 Python 解釋器和運行環境相關的功能。
# sys.path 是一個列表，包含了 Python 解釋器在導入模組時會搜尋的路徑。
# 當你使用 import 語句導入模組時 Python 會依次在 sys.path 中的路徑下尋找對應的模組文件。
sys.path.append("../ecg-classification/")
# sys.path.append("C:\\Users\\Chen_Lab01\\Documents\\GitHub/ecg-classification")
# from IPython.display import Video
# import cv2
from matplotlib import pyplot as plt
plt.style.use("ggplot") #  是 Matplotlib 庫中用於設置繪圖樣式的函數。它使用了一種名為 "ggplot" 的預定義樣式
import matplotlib
matplotlib.use("TkAgg")
                        #  該樣式模仿了 R 語言的 ggplot2 繪圖包的外觀。
# print(sys.path)
from ecg_tools.config import EcgConfig, Mode
from ecg_tools.data_loader import DatasetConfig, get_data_loaders
from ecg_tools.model import ECGformer
from ecg_tools.train import ECGClassifierTrainer


Load model

In [2]:
import torch
config = EcgConfig()    
model_quantized = torch.load("..\\model_save\\model_quantized.pth")
model = torch.load("..\\model_save\\model.pth")

C:\Users\Xaio\anaconda3\envs\pytorch-ecg\lib\site-packages\torch\_utils.py:382: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  device=storage.device,


In [3]:
import einops
loader = get_data_loaders(DatasetConfig())
accuracy = 0
for signal, label in loader[Mode.eval]:
    p = model(einops.rearrange(signal, "b c e -> b e c"))
    print(p)
    print(label)
    # print(signal.shape, label.shape)
    print(p.argmax(1) == label)
    accuracy += torch.sum(p.argmax(1) == label)
    print(f"accuracy: {accuracy / config.dataset.batch_size}")
    break



tensor([[-0.4664, -0.5813, -0.0759,  0.8808, -0.3671, -0.7415],
        [-0.4398, -0.5792, -0.0414,  0.8643, -0.3161, -0.7562],
        [-0.4712, -0.5695, -0.0611,  0.8864, -0.3544, -0.7355],
        [-0.4462, -0.5696, -0.0674,  0.8759, -0.3429, -0.7418],
        [-0.4285, -0.5792, -0.0772,  0.8672, -0.3274, -0.7411],
        [-0.3981, -0.5828, -0.0495,  0.8423, -0.2698, -0.7635],
        [-0.4494, -0.5558, -0.0697,  0.8976, -0.3636, -0.7337],
        [-0.4550, -0.5648, -0.0774,  0.8840, -0.3715, -0.7420],
        [-0.4529, -0.5736, -0.0935,  0.9085, -0.3562, -0.7340],
        [-0.4061, -0.5896, -0.0596,  0.8443, -0.2862, -0.7518],
        [-0.4213, -0.5731, -0.0697,  0.8633, -0.2930, -0.7251],
        [-0.4356, -0.5898, -0.0624,  0.8786, -0.3294, -0.7417],
        [-0.4546, -0.5911, -0.0715,  0.8920, -0.3542, -0.7540],
        [-0.4694, -0.5625, -0.0958,  0.9038, -0.3837, -0.7250],
        [-0.4100, -0.5911, -0.0479,  0.8370, -0.2743, -0.7606],
        [-0.2910, -0.6222, -0.0295,  0.7

## Parameter 提取

In [5]:
for name in model_quantized.state_dict():
   print(name)

print(model_quantized.state_dict()['encoder.0.0.block.1.queries_projection._packed_params._packed_params'])

positional_encoding
encoder.0.0.block.0.weight
encoder.0.0.block.0.bias
encoder.0.0.block.1.queries_projection.scale
encoder.0.0.block.1.queries_projection.zero_point
encoder.0.0.block.1.queries_projection._packed_params.dtype
encoder.0.0.block.1.queries_projection._packed_params._packed_params
encoder.0.0.block.1.values_projection.scale
encoder.0.0.block.1.values_projection.zero_point
encoder.0.0.block.1.values_projection._packed_params.dtype
encoder.0.0.block.1.values_projection._packed_params._packed_params
encoder.0.0.block.1.keys_projection.scale
encoder.0.0.block.1.keys_projection.zero_point
encoder.0.0.block.1.keys_projection._packed_params.dtype
encoder.0.0.block.1.keys_projection._packed_params._packed_params
encoder.0.0.block.1.final_projection.scale
encoder.0.0.block.1.final_projection.zero_point
encoder.0.0.block.1.final_projection._packed_params.dtype
encoder.0.0.block.1.final_projection._packed_params._packed_params
encoder.0.1.block.0.weight
encoder.0.1.block.0.bias
enco